In [0]:
%sql
CREATE TABLE delta.`/tmp/deltain-table1` USING DELTA AS SELECT col1 as id FROM VALUES 200,300,400,500,600;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT * FROM delta.`/tmp/deltain-table1`;

id
200
300
400
500
600


In [0]:
streamingDf = spark.readStream.format("rate").load()
stream = streamingDf.selectExpr("value as id").writeStream.format("delta").option("checkpointLocation", "/tmp/checkpoint").start("/tmp/delta-table1")

In [0]:
%sql
UPDATE delta.`/tmp/delta-table1` SET id = id + 100 WHERE id % 2 == 0;


DELETE FROM delta.`/tmp/delta-table1` WHERE id % 2 == 0;


CREATE TEMP VIEW newData AS SELECT col1 AS id FROM VALUES 1,3,5,7,9,11,13,15,17,19;

MERGE INTO delta.`/tmp/delta-table1` AS oldData
USING newData
ON oldData.id = newData.id
WHEN MATCHED
  THEN UPDATE SET id = newData.id
WHEN NOT MATCHED
  THEN INSERT (id) VALUES (newData.id);

SELECT * FROM delta.`/tmp/delta-table1`;

id
21
25
29
33
37
41
45
49
9
17


In [0]:
stream2 = spark.readStream.format("delta").load("/tmp/delta-table1").writeStream.format("console").start()